In [1]:
import pandas as pd

## Data Collection

In [2]:
df_poi_per_district = pd.read_pickle("./pickles/savedDF_POI_per_district.pickle")

In [3]:
df_result_taxi = pd.read_pickle("./pickles/savedDF_taxi_per_district.pkl")

In [4]:
df_green_spaces = pd.read_pickle("./pickles/savedDF_green_spaces.pkl")

In [5]:
df_mobike = pd.read_pickle("./pickles/savedDF_mobike.pkl")

In [6]:
df_poi_per_district['Taxi'] = df_result_taxi['nb_taxi']
df_poi_per_district['Green_spaces'] = df_green_spaces['green_area_proportion']
df_poi_per_district['mobike'] = df_mobike['nb_mobike']

In [7]:
target = pd.read_pickle("./pickles/TARGET-communities-extract-wellbeing.pkl")
header = target[0]
df_target = pd.DataFrame(data=target, columns=header)
#df_target.head()

In [8]:
df_target = df_target.drop(df_target.index[0])

# Conversion

In [9]:
df_target = df_target[df_target['clean']!= '']
df_target = df_target[df_target['smell']!= '']
df_target = df_target[df_target['noise']!= '']
df_target = df_target[df_target['longitude']!= '']
df_target = df_target[df_target['latitude']!= '']


df_target['clean'] = df_target['clean'].apply(lambda x: float(x))
df_target['smell'] = df_target['smell'].apply(lambda x: float(x))
df_target['noise'] = df_target['noise'].apply(lambda x: float(x))
df_target['longitude'] = df_target['longitude'].apply(lambda x: float(x))
df_target['latitude'] = df_target['latitude'].apply(lambda x: float(x))

In [10]:
df_target['smell'] = df_target['smell'].apply(lambda x: 1 if x==0  else 0)
df_target['noise'] = df_target['noise'].apply(lambda x: 1 if x==0  else 0)
#df_target.head()

In [11]:
df_target['mean'] = (df_target['clean']*1 + df_target['noise']*1+ df_target['smell']*1)/3
#df_target.head()

# Point geo pour chaque lieux

In [12]:
from shapely.geometry import Point # Shapely for converting latitude/longtitude to geometry
import geopandas as gpd

In [13]:
df_ML = df_target[['mean','longitude','latitude']]

In [14]:
geometry = [Point(xy) for xy in zip(df_ML['longitude'], df_ML['latitude'])]

In [15]:
df_ML['geometry'] = geometry

C:\Users\sebr\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
#df_ML.head()

In [17]:
gdf = gpd.GeoDataFrame(df_ML, geometry=df_ML.geometry)

In [18]:
#gdf.plot()

## Check if the point is included in polygon

In [19]:
happiness = {}
total = 0
sum_mean = 0
for index, polygon in df_poi_per_district['geometry'].items():
    for mean, point in zip(df_ML['mean'], df_ML['geometry']):
        if point.within(polygon) == True:
            sum_mean += mean
            total += 1
    if total != 0 : 
        happiness[index] = sum_mean/total
    else:
        happiness[index] = 0
    sum_mean = 0
    total = 0

In [20]:
df_poi_per_district['happiness'] = happiness.values()

In [21]:
#df_poi_per_district

In [22]:
df_poi_per_district.columns

Index(['NAME', 'TYPE', 'geometry', 'restaurant', 'Bicycle_park', 'Bus',
       'Railway_station_entrance_exit', 'Convenience Store', 'Scenic_spot',
       'Sport', 'Taxi', 'Green_spaces', 'mobike', 'happiness'],
      dtype='object')

## Scale the datas

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_poi_per_district['restaurant_scale'] = scaler.fit_transform(df_poi_per_district[['restaurant']])
df_poi_per_district['Bicycle_park_scale'] = scaler.fit_transform(df_poi_per_district[['Bicycle_park']])
df_poi_per_district['Bus_scale'] = scaler.fit_transform(df_poi_per_district[['Bus']])
df_poi_per_district['Railway_scale'] = scaler.fit_transform(df_poi_per_district[['Railway_station_entrance_exit']])
df_poi_per_district['Convenience_store_scale'] = scaler.fit_transform(df_poi_per_district[['Convenience Store']])
df_poi_per_district['Scenic_spot_scale'] = scaler.fit_transform(df_poi_per_district[['Scenic_spot']])
df_poi_per_district['Sport_scale'] = scaler.fit_transform(df_poi_per_district[['Sport']])
df_poi_per_district['Taxi_scale'] = scaler.fit_transform(df_poi_per_district[['Taxi']])
df_poi_per_district['Green_spaces_scale'] = scaler.fit_transform(df_poi_per_district[['Green_spaces']])
df_poi_per_district['mobike_scale'] = scaler.fit_transform(df_poi_per_district[['mobike']])

In [24]:
df_poi_per_district.to_pickle("savedDF_datasets_ML.pkl")